In [31]:
import pandas as pd
import numpy as np
import os
import glob
pd.set_option("display.max_colwidth", 1000)

In [120]:
# re-format junction sites
bam = 'ENCFF468DJE.bam'
wd = '/home/elvannostrand/scratch/encode_rnaseq_psis_20170714tophat/'
out_bed = '/home/bay001/projects/encode/analysis/tests/eric_jxc_tests/{}.bed'.format(bam)
bam_to_test = os.path.join(wd,'{}.primary.namesort.bam.ATAC.psi'.format(bam))
eric = pd.read_table(bam_to_test, names=['pos','exc','inc','dpsi'])
eric['chrom'] = eric['pos'].apply(lambda x: x.split(':')[0])
eric['start'] = eric['pos'].apply(lambda x: int(x.split(':')[2].split('-')[1].split('|')[0]))
eric['end'] = eric['pos'].apply(lambda x: int(x.split(':')[2].split('|')[1].split('-')[0]))
eric['strand'] = eric['pos'].apply(lambda x: x.split(':')[1])
eric['pos2'] = eric['pos'].apply(lambda x: x.split('ENST')[0][:-1])
# eric = eric[eric.duplicated(['exc','inc','dpsi','chrom','start','end','strand','pos2'])]
# eric = eric.drop_duplicates(['exc','inc','dpsi','chrom','start','end','strand','pos2'])
print(eric.shape)
eric.reset_index(inplace=True)
# eric = eric[['chrom','start','end','index','exc','strand']]
# eric.to_csv(out_bed, sep='\t', index=False, header=False)
# eric.head()

(261, 9)


In [159]:
# use my caller to compare

out_file = '/home/bay001/projects/encode/analysis/tests/eric_jxc_tests/{}.jxc'.format(bam)
bam_dir = '/home/bay001/projects/encode/analysis/tests/eric_jxc_tests'

cmd = 'python /home/bay001/projects/codebase/junction-counter/junction_counter/count_junctions.py '
cmd = cmd + '--bed {} '.format(out_bed)
cmd = cmd + '--bam {} '.format(os.path.join(bam_dir,bam))
cmd = cmd + '--min_overlap 10 '
cmd = cmd + '--jxc_only '
cmd = cmd + '--outfile {} '.format(out_file)
cmd = cmd + '--report_names'
! $cmd

100%|#########################################| 261/261 [00:06<00:00, 38.22it/s]


In [148]:
brian = pd.read_table(out_file, index_col=0)
brian.head(10)

,jxn,skip_depth,incl_depth,three_prime_incl_depth,five_prime_incl_depth
0,chr16:3533597-3534698:+,89,21,11,10
1,chr7:30661116-30661932:+,54,1,1,0
2,chr15:66169705-66170149:+,0,357,122,237
3,chr19:39334744-39336289:-,151,21,7,14
4,chr11:73914866-73915397:+,34,0,0,0
5,chr6:43496663-43498425:-,45,8,5,3
6,chr8:145139778-145139945:+,13,17,8,9
7,chr11:74676968-74680609:+,95,0,0,0
8,chr16:24835241-24835823:+,0,100,29,71
9,chr1:24301567-24305202:-,68,53,24,29


In [131]:
merged = pd.merge(eric, brian, how='left', left_index=True, right_index=True)
merged = merged[['chrom','start','end','strand','exc','skip_depth']].sort_values(by='exc')
merged[merged['exc']!=merged['skip_depth']].drop_duplicates()

,chrom,start,end,strand,exc,skip_depth
209,chr18,77726678,77728088,+,8,9
100,chr8,21846594,21847854,+,31,33
124,chr2,220091655,220092491,-,33,51
141,chr17,42758011,42759370,-,33,43
136,chr19,1218499,1219322,+,47,67
84,chr7,150751552,150752113,-,51,82
214,chr6,159049522,159050761,+,73,76
219,chr10,135209281,135209666,+,78,101
13,chr19,13260407,13260541,-,103,105
200,chr16,66967969,66968125,-,135,155


In [124]:
eric[eric['start']==220091655]

,index,pos,exc,inc,dpsi,chrom,start,end,strand,pos2
16,16,chr2:-:220091590-220091655|220092491-220092775:ENST00000486766.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092775
22,22,chr2:-:220091590-220091655|220092491-220092535:ENST00000432520.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
53,53,chr2:-:220091590-220091655|220092491-220092535:ENST00000361242.4,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
60,60,chr2:-:220091590-220091655|220092491-220092535:ENST00000439812.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
75,75,chr2:-:220091590-220091655|220092491-220092535:ENST00000457841.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
88,88,chr2:-:220091590-220091655|220092491-220092535:ENST00000396761.2,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
120,120,chr2:-:220091590-220091655|220092491-220092535:ENST00000436856.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
124,124,chr2:-:220091590-220091655|220092491-220092535:ENST00000466217.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
138,138,chr2:-:220091590-220091655|220092491-220092535:ENST00000409422.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535
157,157,chr2:-:220091590-220091655|220092491-220092535:ENST00000434939.1,33,6,0.08333,chr2,220091655,220092491,-,chr2:-:220091590-220091655|220092491-220092535


In [101]:
def get_list_of_names(ix, df=df, col='skip_names'):
    return df.ix[ix][col].split(',')

In [103]:
get_list_of_names(209, brian, 'skip_names')

['DF8F08P1:365:C60G9ACXX:5:2305:19357:61656',
 'DF8F08P1:365:C60G9ACXX:5:1115:17224:40497',
 'DF8F08P1:365:C60G9ACXX:5:2205:16094:78233',
 'DF8F08P1:365:C60G9ACXX:5:1316:2026:13260',
 'DF8F08P1:365:C60G9ACXX:5:2309:4185:44281',
 'DF8F08P1:365:C60G9ACXX:5:2316:9426:27614',
 'DF8F08P1:365:C60G9ACXX:5:2314:4336:42253',
 'DF8F08P1:365:C60G9ACXX:5:2102:4410:4417',
 'DF8F08P1:365:C60G9ACXX:5:2214:5220:76935']

In [ ]:
df2 = pd.read_table('/home/bay001/projects/encode/analysis/tests/eric_jxc_tests/all_jxc/ENCFF756RDZ.bam.jxc', index_col=0)
df2.head()

In [ ]:
merged = pd.merge(df, df2, how='outer', left_index=True, right_index=True)
merged[merged['depth']!=merged['skip_depth']]

In [ ]:
import pysam

In [ ]:
x = pysam.AlignedSegment()
x.cigartuples = [(0, 1)]
x

In [ ]:
st = 'D80KHJN1:241:C5HF7ACXX:6:1314:9778:90604	83	chr16	83842300	50	52M558N48M	=	83841691	-1267	CAGCAGATGCAAGATAAATTTCAGACCATGTCTGACCAGATCATTGGGAGAATTGATGATATGAGTAGTCGCATTGATGATCTGGAAAAGAATATCGCGG	DDDDDDEDDDFDDEDDEEEEEEEDFFFFGHFHGHHJJJJJIJJIJJJJJJJIGGJJJJJJJIJJJJJJJJJJJJJJJJJJJJJJJJJHHHHHFFFFFCCC	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:100	YT:Z:UU	XS:A:+	NH:i:1'
s = pysam.()